Context awareness for avaoiding false positives.  

In [ ]:
pip install rapidfuzz

**V-1**  

In [ ]:
import nltk
import re
import pandas as pd
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet, stopwords
from rapidfuzz import process
from collections import defaultdict
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

# Ensure necessary NLTK resources are downloaded
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Initialize NLP tools
STOP_WORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Load dictionaries
def load_dictionary(dictionary_path):
    """Load English dictionary words"""
    df = pd.read_csv(dictionary_path, header=None)
    return set(df[0].astype(str).str.lower())

def load_profanity_words(csv_file):
    """Load and preprocess profanity dictionary"""
    profanity_df = pd.read_csv(csv_file, encoding="latin-1")
    profanity_df["Text"] = profanity_df["Text"].str.replace(r"[\x92\x91\x96\x97]", "'", regex=True)
    single_words = {}
    phrases = defaultdict(list)
    for _, row in profanity_df.iterrows():
        text = row["Text"].lower()
        if ' ' in text:  # It's a phrase
            phrases[len(text.split())].append((text, row["Category"]))
        else:
            single_words[text] = row["Category"]
    return single_words, phrases

# Load dictionaries
dictionary_path = "english_dictionary.csv"
profanity_path = "semantically_labeled_profanities.csv"
english_words = load_dictionary(dictionary_path)
profanity_dict, profanity_phrases = load_profanity_words(profanity_path)

def deobfuscate_word(word):
    """Convert leetspeak and symbols back to letters with profanity priority"""
    profanity_patterns = {
        r'f[!@#$%^&*.]*u[!@#$%^&*.]*c[!@#$%^&*.]*k': 'fuck',
        r's[!@#$%^&*.]*h[!@#$%^&*.]*i[!@#$%^&*.]*t': 'shit',
        r'a[!@#$%^&*.]*s[!@#$%^&*.]*s': 'ass',
        r'b[!@#$%^&*.]*i[!@#$%^&*.]*t[!@#$%^&*.]*c[!@#$%^&*.]*h': 'bitch',
        r'c[!@#$%^&*.]*u[!@#$%^&*.]*n[!@#$%^&*.]*t': 'cunt',
        r'd[!@#$%^&*.]*i[!@#$%^&*.]*c[!@#$%^&*.]*k': 'dick',
        r'p[!@#$%^&*.]*u[!@#$%^&*.]*s[!@#$%^&*.]*s[!@#$%^&*.]*y': 'pussy',
    }
    lower_word = word.lower()
    for pattern, replacement in profanity_patterns.items():
        if re.fullmatch(pattern, lower_word):
            return replacement
    replacements = {
        r'[@4]': 'a',
        r'[$5]': 's',
        r'[1!|i]': 'i',
        r'[0°]': 'o',
        r'[3]': 'e',
        r'[7]': 't',
        r'[8]': 'b',
        r'[9]': 'g',
    }
    for pattern, replacement in replacements.items():
        lower_word = re.sub(pattern, replacement, lower_word)
    return lower_word

def custom_tokenize(text):
    """Improved tokenizer that handles contractions and obfuscations"""
    text = re.sub(r"(\w+)'(\w+)", r"\1'\2", text)
    return re.findall(r"[@\w$!']+", text.lower())

def reduce_redundant_letters(word):
    """Handle excessive letter repetition"""
    reduced = re.sub(r'(.)\1{2,}', r'\1\1', word)
    if reduced not in english_words:
        reduced = re.sub(r'(.)\1+', r'\1', reduced)
    return reduced

def normalize_word(word):
    """Normalize word with profanity detection focus"""
    if len(word) <= 2 or word in STOP_WORDS:
        return word
    deobfuscated = deobfuscate_word(word)
    if deobfuscated.endswith('ing'):
        base_form = deobfuscated[:-3]
        if base_form in profanity_dict:
            return base_form
        if f"{base_form}in" in profanity_dict:
            return f"{base_form}in"
    if deobfuscated in profanity_dict:
        return deobfuscated
    closest = process.extractOne(deobfuscated, profanity_dict.keys(), score_cutoff=85)
    if closest:
        return closest[0]
    return deobfuscated

# Context-aware model setup using Toxic-BERT
CONTEXT_MODEL_NAME = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(CONTEXT_MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(CONTEXT_MODEL_NAME)

def analyze_context(text):
    """Use transformer model to detect toxic context"""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    scores = softmax(outputs.logits.detach().numpy()[0])
    # Adjusted indices: model returns 6 scores, indices 0-5.
    return {
        "toxic": scores[0],
        "severe_toxic": scores[1],
        "obscene": scores[2],
        "threat": scores[3],
        "insult": scores[4],
        "identity_hate": scores[5]
    }

def contains_profanity(text):
    """Enhanced profanity detection with transformer-based context awareness"""
    words = custom_tokenize(text)
    flagged_words = {}

    # Check single words using dictionary/fuzzy logic
    for word in words:
        if word in STOP_WORDS or len(word) <= 2:
            continue
        normalized = normalize_word(word)
        if normalized in STOP_WORDS:
            continue
        if normalized in profanity_dict:
            flagged_words[word] = profanity_dict[normalized]

    # Check multi-word phrases (2-4 words)
    for phrase_length in range(2, min(5, len(words) + 1)):
        for i in range(len(words) - phrase_length + 1):
            phrase = ' '.join(words[i:i+phrase_length])
            if phrase in profanity_phrases[phrase_length]:
                for term, category in profanity_phrases[phrase_length]:
                    if term == phrase:
                        flagged_words[phrase] = category
                        break

    # If no profanity flagged by the dictionary logic, return early
    if not flagged_words:
        return "Clean"

    # Run transformer-based context analysis on the original text
    try:
        toxicity_scores = analyze_context(text)
        max_toxicity = max(toxicity_scores.values())
        # Define thresholds (adjust these as needed)
        if max_toxicity < 0.7:
            return "Clean"
        elif max_toxicity < 0.4:
            return "Clean"
        # Special-case: ignore "shit" if overall toxicity is low and certain context words exist
        if "shit" in flagged_words and toxicity_scores['toxic'] < 0.5:
            if any(word in text.lower() for word in ["deep", "happening", "going on"]):
                return "Clean"
    except Exception as e:
        print(f"Context analysis failed: {str(e)}")
        return flagged_words

    return flagged_words

# Example Usage
if __name__ == "__main__":
    test_cases = [
      "You are a f@cking id!0t",
      "h3ll yeah! that's $tupid",
      "Sh1t, you're an @ssh0le!",
      "fucking awesome",
      "This is a clean sentence",
      "I just shit my pants",
      "Go to hell you motherfucker",
      "That's some bullshit right there",
      "What the f*** is this?",
      "You're a dumb@ss"
    ]
    for text in test_cases:
        print(f"Text: {text}")
        result = contains_profanity(text)
        print(f"Result: {result}\n")

Text: You are a f@cking id!0t  
Result: {'f@cking': 'Explicit', 'id!0t': 'Insults & Personal Attacks'}  

Text: h3ll yeah! that's $tupid  
Result: {'$tupid': 'Insults & Personal Attacks'}  

Text: Sh1t, you're an @ssh0le!  
Result: {'sh1t': 'Insults & Personal Attacks', '@ssh0le!': 'Explicit'}  

Text: fucking awesome  
Result: Clean  

Text: This is a clean sentence  
Result: Clean  

Text: I just shit my pants  
Result: {'shit': 'Insults & Personal Attacks'}  

Text: Go to hell you motherfucker  
Result: {'motherfucker': 'Explicit'}  

Text: That's some bullshit right there  
Result: Clean  

Text: What the f*** is this?  
Result: Clean  

Text: You're a dumb@ss  
Result: {'dumb@ss': 'Insults & Personal Attacks'}  

**V-2**

In [ ]:
import nltk
import re
import pandas as pd
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet, stopwords
from rapidfuzz import process
from collections import defaultdict
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

# Ensure necessary NLTK resources are downloaded
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Initialize NLP tools
STOP_WORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Load dictionaries
def load_dictionary(dictionary_path):
    """Load English dictionary words"""
    df = pd.read_csv(dictionary_path, header=None)
    return set(df[0].astype(str).str.lower())

def load_profanity_words(csv_file):
    """Load and preprocess profanity dictionary"""
    profanity_df = pd.read_csv(csv_file, encoding="latin-1")
    profanity_df["Text"] = profanity_df["Text"].str.replace(r"[\x92\x91\x96\x97]", "'", regex=True)
    single_words = {}
    phrases = defaultdict(list)
    for _, row in profanity_df.iterrows():
        text = row["Text"].lower()
        if ' ' in text:  # It's a phrase
            phrases[len(text.split())].append((text, row["Category"]))
        else:
            single_words[text] = row["Category"]
    return single_words, phrases

# Load dictionaries
dictionary_path = "english_dictionary.csv"
profanity_path = "semantically_labeled_profanities.csv"
english_words = load_dictionary(dictionary_path)
profanity_dict, profanity_phrases = load_profanity_words(profanity_path)

def deobfuscate_word(word):
    """Convert leetspeak and symbols back to letters with profanity priority"""

    lower_word = word.lower()
    # Special-case: if the word is like f*** (an "f" followed by at least two asterisks)
    if re.fullmatch(r"f\*{2,}", lower_word):
        return "fuck"
    profanity_patterns = {
        r'f[!@#$%^&*.]*u[!@#$%^&*.]*c[!@#$%^&*.]*k': 'fuck',
        r's[!@#$%^&*.]*h[!@#$%^&*.]*i[!@#$%^&*.]*t': 'shit',
        r'a[!@#$%^&*.]*s[!@#$%^&*.]*s': 'ass',
        r'b[!@#$%^&*.]*i[!@#$%^&*.]*t[!@#$%^&*.]*c[!@#$%^&*.]*h': 'bitch',
        r'c[!@#$%^&*.]*u[!@#$%^&*.]*n[!@#$%^&*.]*t': 'cunt',
        r'd[!@#$%^&*.]*i[!@#$%^&*.]*c[!@#$%^&*.]*k': 'dick',
        r'p[!@#$%^&*.]*u[!@#$%^&*.]*s[!@#$%^&*.]*s[!@#$%^&*.]*y': 'pussy',
    }
    lower_word = word.lower()
    for pattern, replacement in profanity_patterns.items():
        if re.fullmatch(pattern, lower_word):
            return replacement
    replacements = {
        r'[@4]': 'a',
        r'[$5]': 's',
        r'[1!|i]': 'i',
        r'[0°]': 'o',
        r'[3]': 'e',
        r'[7]': 't',
        r'[8]': 'b',
        r'[9]': 'g',
    }
    for pattern, replacement in replacements.items():
        lower_word = re.sub(pattern, replacement, lower_word)
    return lower_word

def custom_tokenize(text):
    """Improved tokenizer that handles contractions and obfuscations"""
    text = re.sub(r"(\w+)'(\w+)", r"\1'\2", text)
    return re.findall(r"[@\w$!']+", text.lower())

def reduce_redundant_letters(word):
    """Handle excessive letter repetition"""
    reduced = re.sub(r'(.)\1{2,}', r'\1\1', word)
    if reduced not in english_words:
        reduced = re.sub(r'(.)\1+', r'\1', reduced)
    return reduced

def normalize_word(word):
    """Normalize word with profanity detection focus"""
    if len(word) <= 2 or word in STOP_WORDS:
        return word
    deobfuscated = deobfuscate_word(word)
    if deobfuscated.endswith('ing'):
        base_form = deobfuscated[:-3]
        if base_form in profanity_dict:
            return base_form
        if f"{base_form}in" in profanity_dict:
            return f"{base_form}in"
    if deobfuscated in profanity_dict:
        return deobfuscated
    closest = process.extractOne(deobfuscated, profanity_dict.keys(), score_cutoff=85)
    if closest:
        return closest[0]
    return deobfuscated

# Context-aware model setup using Toxic-BERT
CONTEXT_MODEL_NAME = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(CONTEXT_MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(CONTEXT_MODEL_NAME)

def analyze_context(text):
    """Use transformer model to detect toxic context"""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    scores = softmax(outputs.logits.detach().numpy()[0])
    # Adjusted indices: model returns 6 scores, indices 0-5.
    return {
        "toxic": scores[0],
        "severe_toxic": scores[1],
        "obscene": scores[2],
        "threat": scores[3],
        "insult": scores[4],
        "identity_hate": scores[5]
    }

def contains_profanity(text):
    """Enhanced profanity detection with transformer-based context awareness and special-case filtering"""
    words = custom_tokenize(text)
    flagged_words = {}

    # Check single words using dictionary/fuzzy logic
    for word in words:
        if word in STOP_WORDS or len(word) <= 2:
            continue
        normalized = normalize_word(word)
        if normalized in STOP_WORDS:
            continue
        if normalized in profanity_dict:
            flagged_words[word] = profanity_dict[normalized]

    # Check multi-word phrases (2-4 words)
    for phrase_length in range(2, min(5, len(words) + 1)):
        for i in range(len(words) - phrase_length + 1):
            phrase = ' '.join(words[i:i+phrase_length])
            if phrase in profanity_phrases[phrase_length]:
                for term, category in profanity_phrases[phrase_length]:
                    if term == phrase:
                        flagged_words[phrase] = category
                        break

    # If no profanity flagged by the dictionary logic, return early
    if not flagged_words:
        return "Clean"

    # Run transformer-based context analysis on the original text
    try:
        toxicity_scores = analyze_context(text)
        max_toxicity = max(toxicity_scores.values())
        # Define thresholds (adjust as needed)
        if max_toxicity < 0.7 or max_toxicity < 0.4:
            return "Clean"
        # Special-case: ignore "shit" if overall toxicity is low and context words exist
        if "shit" in flagged_words:
            # Example check: if the text contains "pants", we assume it's literal
            if "pants" in text.lower() or "poop" in text.lower():
                flagged_words.pop("shit")
    except Exception as e:
        print(f"Context analysis failed: {str(e)}")
        return flagged_words

    return flagged_words if flagged_words else "Clean"


# Example Usage
if __name__ == "__main__":
    test_cases = [
      "You are a f@cking id!0t",
      "h3ll yeah! that's $tupid",
      "Sh1t, you're an @ssh0le!",
      "fucking awesome",
      "This is a clean sentence",
      "I just shit my pants",
      "Go to hell you motherfucker",
      "f*** you",
      "That's some bullshit right there",
      "What the f*** is this?",
      "You're a dumb@ss"
    ]
    for text in test_cases:
        print(f"Text: {text}")
        result = contains_profanity(text)
        print(f"Result: {result}\n")


Text: You are a f@cking id!0t  
Result: {'f@cking': 'Explicit', 'id!0t': 'Insults & Personal Attacks'}  

Text: h3ll yeah! that's $tupid  
Result: {'$tupid': 'Insults & Personal Attacks'}  

Text: Sh1t, you're an @ssh0le!  
Result: {'sh1t': 'Insults & Personal Attacks', '@ssh0le!': 'Explicit'}  

Text: fucking awesome  
Result: Clean  

Text: This is a clean sentence  
Result: Clean  

Text: I just shit my pants  
Result: Clean  

Text: Go to hell you motherfucker  
Result: {'motherfucker': 'Explicit'}  

Text: f*** you  
Result: Clean  

Text: That's some bullshit right there  
Result: Clean  

Text: What the f*** is this?  
Result: Clean  

Text: You're a dumb@ss  
Result: {'dumb@ss': 'Insults & Personal Attacks'}  